# Clean data

## Sessionn aufbauen & einlesen

In [1]:
hdfs_GrundPfad  = '/bigdata/'
nb_Pfad         = '/home/alfa/BDS_Source/timsSparkSessions/'
DatenPfad       = '/home/alfa/BDS_Source/timsSparkSessions/daten/'

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession

In [3]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[4]") \
      .appName("SparkCSV") \
      .getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.35.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2025-02-07 14:32:13,853 WARN util.Utils: Your hostname, Tims-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
2025-02-07 14:32:13,854 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Einlesen einer CSV-Datei aus dem HDFS
hdfs_csv = "hdfs://" + hdfs_GrundPfad + "movies_hdfs.csv"

# Use header record for column names
df = spark.read \
           .option("header",True) \
           .csv(hdfs_csv, escape='"', inferSchema=True)

# Transform release_date to date

from pyspark.sql.functions import to_date

df = df.withColumn("release_date", to_date(df["release_date"]))

## Df erstellen

Wir wollen jetzt das df filtern: nur Filme, deren vote_count > 50 ist. Ab da beginnen die etwas größeren Filme, mit teilweise >1 Mio Umsatz.

In [17]:
from pyspark.sql.functions import col

df_newbig = df.filter((col("vote_count") > 50))

In [18]:
print(df_newbig.count())
df_newbig.printSchema()

27689
root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- backdrop_path: string (nullable = true)
 |-- budget: integer (nullable = true)
 |-- homepage: string (nullable = true)
 |-- imdb_id: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- poster_path: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- keywords: string (

Es gibt einige unnötige Spalten: 

backdrop_path
imdb_id
overview
poster_path

Die sind groß und bringen kaum Mehrwert. Rauswerfen!

In [19]:
# List of columns to drop
columns_to_drop = ['backdrop_path', 'imdb_id', 'overview', 'poster_path']

# Drop the columns from the DataFrame
df_newbig = df_newbig.drop(*columns_to_drop)

print(df_newbig.count())
df_newbig.printSchema()

27689
root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- homepage: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- tagline: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- keywords: string (nullable = true)



>spark_df in pandas_df umwandeln und daraus csv erstellen

In [20]:
# Convert Spark DataFrame to pandas DataFrame
pandas_df = df_newbig.toPandas()

# Display the pandas DataFrame
print(pandas_df.head())

       id            title  vote_average  vote_count    status release_date  \
0   27205        Inception         8.364       34495  Released   2010-07-15   
1  157336     Interstellar         8.417       32571  Released   2014-11-05   
2     155  The Dark Knight         8.512       30619  Released   2008-07-16   
3   19995           Avatar         7.573       29815  Released   2009-12-15   
4   24428     The Avengers         7.710       29166  Released   2012-04-25   

      revenue  runtime  adult     budget  \
0   825532764      148  False  160000000   
1   701729206      169  False  165000000   
2  1004558444      152  False  185000000   
3  2923706026      162  False  237000000   
4  1518815515      143  False  220000000   

                                         homepage original_language  \
0     https://www.warnerbros.com/movies/inception                en   
1               http://www.interstellarmovie.net/                en   
2  https://www.warnerbros.com/movies/dark-knigh

In [22]:
# Convert pandas DataFrame to CSV and save to the specified directory
pandas_df.to_csv('./daten/movies.csv', index=False)

# Display a message indicating the file has been saved
print("CSV file has been saved to './daten/movies.csv'")

CSV file has been saved to './daten/movies.csv'
